In [ ]:
# from utils import *
import requests
import logging
import os
# from time import time
import time
from queue import Queue
from threading import Thread
# from crawler_product import *
# from kafka_producer import produce
import pandas as pd 
from kafka import KafkaProducer
import json
from crawl_tiki_products.utils import *
import random
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
bootstrap_servers = ['kafka-1:9092', 'kafka-2:9092', 'kafka-3:9092']
topicName = 'tiki'

producer = KafkaProducer(bootstrap_servers = bootstrap_servers,
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [ ]:
parentCategory = ["1789","4221","1815","1846","1801","4384","2549","1520","1975","8594","8322"]

# parentCategory = ["1815","1801"]
cate_url = "https://tiki.vn/api/v2/products?limit=48&include=advertisement&aggregations=1&category={cateid}&page={page}"
product_url = "https://tiki.vn/api/v2/products/{}"
flatten_field = [ "badges", "inventory", "categories", "rating_summary", 
                      "brand", "seller_specifications", "current_seller", "other_sellers", 
                      "configurable_options",  "configurable_products", "specifications", "product_links",
                      "services_and_promotions", "promotions", "stock_item", "installment_info" ]

In [ ]:
def retryRequest(url):

    headers = random.choice(headers_list)
    # print(user_agent)
    session = requests.Session()
    retry = Retry(connect=4, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(url, headers=headers)

    return response

In [ ]:
topic = "tiki"
def crawl_product_id(url,cateid):
    product_list = []
    products = []
    i = 1
    flag = True
    try:
        os.makedirs(f"./data/raw_product/{cateid}")
    except:
        print("Folder exists")
    while (flag):
        print("Crawl page: ", i)
        print(url.format(cateid = cateid,page = i))
        try:
            response = retryRequest(url.format(cateid = cateid,page = i))
        except Exception as e:
            print(e)
            break
        print(response.status_code)
        if (response.status_code != 200):
            break

        products = json.loads(response.text)["data"]
        last_page = json.loads(response.text)["paging"]["last_page"]
        if (i >= last_page):
            flag = False
        with open(f"./data/raw_product/{cateid}/{cateid}-{i}.json","w+") as f:
            json.dump(products,f,indent = 4,ensure_ascii = False)
        if len(products) == 0:
            break
        i += 1
        try:
            for product in products:
                producer.send(topic,product)
                producer.flush()        
        except Exception as e:
            print(e)

    return product_list,products, i

In [ ]:

class ProductWorker(Thread):

    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue

    def run(self):
        while True:
            # Get the work from the queue and expand the tuple
            cate_id = self.queue.get()
            try:
                crawl_product_id(cate_url,cate_id)
            finally:
                self.queue.task_done()

In [ ]:
def multiThreadScraper(ThreadObject,argList):
    # product_list = load_raw_product("./data/product_id/{}.txt".format("1789"))
    # Create a queue to communicate with the worker threads
    queue = Queue()
    # Create 4 worker threads
    for x in range(4):
        worker = ThreadObject(queue)
        # Setting daemon to True will let the main thread exit even though the workers are blocking
        worker.daemon = True
        worker.start()
    # Put the tasks into the queue as a tuple
    for id in argList:
        # logger.info('Queueing {}'.format(id))
        queue.put((id))
        # product_detail_list.append(product)
    # Causes the main thread to wait for the queue to finish processing all the tasks
    queue.join()

    # logging.info('Took %s seconds', time() - ts)

In [ ]:
multiThreadScraper(ProductWorker,parentCategory)
